In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import json
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [40]:
class PlantDiseasesDataset(Dataset):
    def __init__(self, data_dir, transform=None, data_type = "train"):
        self.data_dir = os.path.join(data_dir, data_type)
        self.transform = transform
        self.data_type = data_type
        self.classes = [d for d in os.listdir(self.data_dir) if os.path.isdir(os.path.join(self.data_dir, d))]
        self.classes.sort()

        self.images = []
        self.labels = []

        for class_idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(self.data_dir, class_name)
            image_files = [f for f in os.listdir(class_dir) if f.endswith(('.JPG','.JPEG', '.PNG', '.jpg', '.jpeg', '.png'))]

            for img_file in image_files:
                self.images.append(os.path.join(class_dir, img_file))
                self.labels.append({
                    'class_idx': class_idx,
                    'class_label': class_name
                })

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]

        try: 
            img = Image.open(img_path).convert('RGB')
            if self.transform:
                img = self.transform(img)

            return {
                'image': img,
                'label': label,
                'path': img_path,
                'type': self.data_type
            }
        except Exception as e:
            print(f"Error loading image: {img_path}: {e}")
            img = torch.zeros((3, 224, 224))
            return {
                'image': img,
                'label': label,
                'path': img_path,
                'type': self.data_type
            }

In [71]:
def extract_emeddings(model, dataset, device, batch_size = 32):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle = False, num_workers = 4)
    result = []
    model.eval()
    # l = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Extracting the embeddings for {dataset.data_type}"):
            images = batch['image'].to(device)
            labels = batch['label']
            features = model(images)
            for i, embedding in enumerate(features.cpu().numpy()):
                class_name = labels['class_label'][i]
                embed = []
                for j in embedding:
                    embed.extend(j[0])
                result.append({
                    'name': class_name,
                    'embeddings':embed,
                    'type': dataset.data_type
                })
            # l+=1

    return result

In [72]:
data_dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/"
if not os.path.exists(data_dir):
    print(f"Dataset dir not found")

transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = models.resnet50(weights='IMAGENET1K_V2')
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.to(device)

all_results = []
data_type = "train"
dataset = PlantDiseasesDataset(data_dir, transform = transform, data_type = data_type)
print(f"Processing {data_type} dataset with {len(dataset)} images and {len(dataset.classes)} classes")
results = extract_emeddings(model, dataset, device)


Using device: cpu
Processing train dataset with 70295 images and 38 classes


Extracting the embeddings for train: 100%|██████████| 2197/2197 [3:06:19<00:00,  5.09s/it]  


In [70]:
len(results)

32

In [74]:
len(results)

70295

In [75]:
output_file = 'plant_disease_embeddings.json'
print(f"Saving embeddings to {output_file}")
with open(output_file, 'w') as f:
    json.dump(results, f)

Saving embeddings to plant_disease_embeddings.json


TypeError: Object of type float32 is not JSON serializable

In [77]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
            return obj.item()  # Convert to native Python type
        return super().default(obj)

In [78]:
output_file = 'plant_disease_embeddings.json'
print(f"Saving embeddings to {output_file}")
with open(output_file, 'w') as f:
    json.dump(results, f, cls=NumpyEncoder)

Saving embeddings to plant_disease_embeddings.json
